<a href="https://colab.research.google.com/github/hoangminhanh/twsm-homework/blob/main/HW5_TextRepresentation(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 7.4 MB/s 
     |████████████████████████████████| 77 kB 6.8 MB/s 
     |████████████████████████████████| 6.6 MB 53.2 MB/s 
     |████████████████████████████████| 880 kB 48.8 MB/s 
     |████████████████████████████████| 596 kB 58.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=ffb2380a9542c795d20524c27a8610079a375c70900225f2c53d8140ab9e1163
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
# Import all the important packages

import pickle
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from scipy.spatial.distance import cosine
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import tensorflow as tf
import torch
from transformers import BertTokenizer, BertModel
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [3]:
# Mount the GG Drive

from google.colab import drive 
drive.mount("/content/drive")

# Importing the Lemmatized data from Google Drive

data_path = "/content/drive/MyDrive/TWSM"
infile = open(data_path+'/Lemma.pkl','rb')
data = pickle.load(infile)

lemma = pd.DataFrame(data, columns=["lemmatized_text"])
lemma.head()

Mounted at /content/drive


,lemmatized_text
0,wonder enlighten car see day door sport car lo...
1,nntp post host carson washington edu fair numb...
2,folks mac plus finally give ghost weekend star...
3,nntp post host amber ssd csd harris com robert...
4,article cowcb world std com tombaker world std...


In [4]:
corpus_gen = [doc.split() for doc in lemma["lemmatized_text"]]

# Part 1: Word2Vec

In [5]:
model = Word2Vec(corpus_gen, size=100, min_count=566) 
# Parameter "size" specifies the the dimensionality of the vector, meaning the vector size 

In [6]:
model.save('word2vec.model')

In [7]:
model.wv.vocab.keys()

dict_keys(['wonder', 'car', 'see', 'day', 'look', 'early', 'call', 'small', 'body', 'know', 'model', 'year', 'history', 'info', 'mail', 'thank', 'nntp', 'post', 'host', 'washington', 'edu', 'number', 'experience', 'send', 'message', 'speed', 'rate', 'add', 'card', 'disk', 'request', 'network', 'base', 'answer', 'guy', 'mac', 'give', 'start', 'life', 'way', 'new', 'machine', 'bit', 'maybe', 'question', 'anybody', 'expect', 'hear', 'suppose', 'access', 'price', 'line', 'like', 'go', 'display', 'probably', 'get', 'feel', 'well', 'great', 'good', 'opinion', 'people', 'use', 'take', 'size', 'money', 'hit', 'real', 'play', 'figure', 'actually', 'advance', 'email', 'news', 'time', 'truth', 'com', 'uucp', 'write', 'article', 'chip', 'far', 'low', 'level', 'stuff', 'pretty', 'require', 'address', 'phone', 'information', 'thing', 'person', 'sense', 'world', 'clear', 'memory', 'error', 'yes', 'quote', 'software', 'check', 'right', 'value', 'set', 'code', 'tell', 'apr', 'john', 'power', 'weapon', 

In [8]:
model.wv['car']

# Showing the vector for the word 'car'

array([ 1.4911838 ,  0.27105722, -1.4270626 ,  0.6491862 , -0.13189852,
       -0.29328054, -1.1664724 , -0.21991941, -1.0196466 , -0.36969563,
        0.9672887 , -0.85468525,  1.35709   ,  1.1705366 ,  0.65667605,
       -1.1745235 ,  1.33018   ,  0.06567831,  0.62622154,  0.21721831,
        0.4494125 , -0.3853249 ,  0.37854084, -0.60643023,  0.14845395,
        0.7716984 ,  0.74779856,  0.261994  ,  0.4342244 , -0.07775514,
        0.35075837, -0.7906314 , -0.40388307,  0.53183556, -1.5614841 ,
        0.2799971 , -0.01042147,  0.7340095 , -0.40339077, -0.7933223 ,
       -0.08589507,  0.50297046, -0.44708347, -1.3901651 ,  0.7212974 ,
        0.23232135, -0.05688187, -1.120604  ,  0.40177825,  0.07601008,
       -1.3912517 ,  0.6709938 , -1.2783272 , -0.5913585 ,  0.1221378 ,
       -0.08490321,  0.9235072 , -0.32661763,  0.22122341, -2.7657511 ,
        1.7677401 , -0.02230113,  0.331445  , -2.2923014 , -0.08860268,
        0.02658519,  0.11294232, -0.84872794,  0.30872205, -0.21

In [9]:
# Show the words that are likely similar to car

model.wv.most_similar('car')

[('bike', 0.6998724937438965),
 ('buy', 0.6064159870147705),
 ('turn', 0.5239871740341187),
 ('get', 0.5017094612121582),
 ('friend', 0.48795849084854126),
 ('pay', 0.48485440015792847),
 ('price', 0.48425495624542236),
 ('figure', 0.4812729060649872),
 ('speed', 0.47923627495765686),
 ('light', 0.47116899490356445)]

In [10]:
model.wv.most_similar(positive=['bike', 'machine'], topn=1)

[('fast', 0.686661958694458)]

In [11]:
print(model.wv.most_similar(positive=['edu', 'woman'], topn=1))
print(model.wv.most_similar(positive=['great', 'money'], topn=1))
print(model.wv.most_similar(positive=['opinion', 'write'], topn=1))

[('john', 0.565531849861145)]
[('deal', 0.8038709163665771)]
[('uucp', 0.7215341329574585)]


Generate a corpus as a list, where for each document the list entry consists of the list of word2vec embeddings of the words in this document. Remove from it the empty documents. Why were those generated? Aggregate the values at a document level (i.e., one embedding per document) by averaging the word embeddings. 

In [12]:
corpus2 = []

for doc in corpus_gen:
  docs = []
  for word in doc:
    if list(model.wv.vocab.keys()).count(word) > 0:
      docs.append(model.wv[word])
    if len(docs) != 0:
      corpus2.append(np.mean(docs, axis=0))

In [13]:
Word2VecModel = pd.DataFrame(corpus2)
Word2VecModel

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.251094,0.229564,-0.151505,0.111964,0.168110,-0.253534,0.046320,-0.269943,-0.779019,0.042001,...,0.271170,0.071123,0.083710,0.058753,0.379020,0.282407,0.498034,0.473872,-0.176473,-0.155580
1,0.251094,0.229564,-0.151505,0.111964,0.168110,-0.253534,0.046320,-0.269943,-0.779019,0.042001,...,0.271170,0.071123,0.083710,0.058753,0.379020,0.282407,0.498034,0.473872,-0.176473,-0.155580
2,0.871139,0.250311,-0.789284,0.380575,0.018106,-0.273407,-0.560076,-0.244931,-0.899333,-0.163847,...,-0.290579,0.364508,1.230414,-0.192241,-0.358709,0.015575,0.161242,-0.461583,-0.759966,-0.107868
3,0.682268,0.213369,-0.698855,0.015675,0.089140,-0.245368,-0.577999,-0.320122,-1.100727,0.026189,...,-0.034674,0.474498,0.778732,-0.306429,-0.075673,-0.042238,0.288156,-0.255211,-0.785995,0.210238
4,0.315724,0.070513,-0.535066,0.149907,0.093312,-0.332891,-0.473222,-0.389201,-1.073397,0.102204,...,-0.207852,0.286048,0.799602,-0.606088,-0.052077,-0.065621,-0.035220,-0.367099,-0.655491,0.139644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1431733,-0.218732,0.154076,-0.393247,0.138099,0.507014,-0.359997,-0.426432,0.197307,-0.182476,-0.372101,...,0.271989,-0.209927,0.072577,-0.437503,0.156518,0.274763,0.423609,0.014913,-0.320573,0.463255
1431734,-0.218732,0.154076,-0.393247,0.138099,0.507014,-0.359997,-0.426432,0.197307,-0.182476,-0.372101,...,0.271989,-0.209927,0.072577,-0.437503,0.156518,0.274763,0.423609,0.014913,-0.320573,0.463255
1431735,-0.273379,0.144365,-0.322036,0.128128,0.425575,-0.312848,-0.324828,0.141425,-0.194524,-0.305694,...,0.276879,-0.151526,0.124989,-0.308961,0.144907,0.335160,0.315662,0.059374,-0.220904,0.371786
1431736,-0.273379,0.144365,-0.322036,0.128128,0.425575,-0.312848,-0.324828,0.141425,-0.194524,-0.305694,...,0.276879,-0.151526,0.124989,-0.308961,0.144907,0.335160,0.315662,0.059374,-0.220904,0.371786


In [14]:
with open('/content/drive/MyDrive/TWSM/w2v_model.pkl', 'wb') as f:
    pickle.dump(Word2VecModel, f)

Find the most similar words to the embedding representing the first document from 10.
Use cosine() to determine the document on the corpus most similar to it

In [15]:
index = np.argmax(model.wv.cosine_similarities(Word2VecModel.iloc[0,:], model.wv.vectors))
print(index)
model.wv.index2word[index]

335


'wonder'

# Part 2: Doc2Vec

Run gensim doc2vec model on corpus_gen (vector_size=100, min_count=566). (Note: Doc2vec requires tagged documents). Determine the embedding representation for the first document as well as the document in the dataset most similar to it. Compare it to the result in 10. Generate the final corpus as a data frame and store it in GoogleDrive as DoctoVecModel.pkl.

In [16]:
# Creating the tagged document

corpus_tagged = []

for i, tokens in enumerate(corpus_gen):
  corpus_tagged.append(TaggedDocument(tokens, [i]))

In [17]:
corpus_tagged[:3]

[TaggedDocument(words=['wonder', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'small', 'addition', 'bumper', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'engine', 'specs', 'year', 'production', 'car', 'history', 'info', 'funky', 'looking', 'car', 'mail', 'thank'], tags=[0]),
 TaggedDocument(words=['nntp', 'post', 'host', 'carson', 'washington', 'edu', 'fair', 'number', 'brave', 'soul', 'upgrade', 'clock', 'oscillator', 'share', 'experience', 'poll', 'send', 'brief', 'message', 'detailing', 'experience', 'procedure', 'speed', 'attain', 'cpu', 'rate', 'speed', 'add', 'card', 'adapters', 'heat', 'sink', 'hour', 'usage', 'day', 'floppy', 'disk', 'functionality', 'floppy', 'especially', 'request', 'summarize', 'day', 'add', 'network', 'knowledge', 'base', 'clock', 'upgrade', 'haven', 'answer', 'poll', 'thank', 'guy', 'kuo', 'guykuo', 'washington', 'edu'], tags=[1]),
 TaggedDocument(words=['folks', 'mac', 'plus', 'f

In [18]:
model_tagged = Doc2Vec(corpus_tagged, vector_size=100, min_count=566)

In [22]:
# Embedding representations (infer vector) of the first document 

inf_vector = model_tagged.infer_vector(corpus_tagged[0][0])

inf_vector

array([ 0.00343389,  0.05433059, -0.0343034 ,  0.02397159, -0.01670114,
       -0.03579214, -0.02394407, -0.10346383, -0.02407631, -0.06662145,
       -0.02115452, -0.01927171,  0.0817785 ,  0.03106043,  0.0981102 ,
       -0.06753811,  0.00577327,  0.05796448,  0.01148386, -0.02031339,
        0.05609358,  0.03103106,  0.02853415,  0.0450988 , -0.04588893,
        0.06154156, -0.01102371, -0.01484922,  0.04037267, -0.06565206,
        0.01656311,  0.04547077,  0.01263468,  0.01129479, -0.00248212,
       -0.06658413, -0.03082949, -0.00124186,  0.01133191, -0.00326324,
       -0.03676914,  0.04329317, -0.00562959, -0.0463122 , -0.04550274,
       -0.00917411, -0.00603131,  0.01553557, -0.05613521,  0.01835759,
        0.03069397,  0.09705859, -0.02355724,  0.00476354,  0.05521968,
       -0.04971575,  0.02490221, -0.04348683, -0.05098116, -0.09985958,
        0.05203902, -0.06488647,  0.0051053 , -0.05141764,  0.03525161,
        0.06276892, -0.00601061, -0.03281502, -0.01759314, -0.05

In [20]:
Doc2VecModel = [model_tagged.infer_vector(doc) for doc, doc_id in corpus_tagged]

In [21]:
with open('/content/drive/MyDrive/TWSM/d2v_model.pkl', 'wb') as f:
    pickle.dump(Doc2VecModel, f)